In [ ]:
#| default_exp ocr_metric

In [ ]:
#| hide
# %reload_ext autoreload
# %autoreload 0


# install (Colab)

In [ ]:
# try: 
#     import fastcore as FC
# except ImportError: 
#     !pip install -q fastcore
# try:
#     import rich
# except ImportError:
#     !pip install -q rich


# Developing a metric for OCR of Comics/Manga texts


# Prologue

In [ ]:
#| export
from __future__ import annotations

import difflib
import html

from IPython.display import display
from IPython.display import HTML
from rich.console import Console


In [ ]:
import fastcore.all as FC
import fastcore.xtras  # patch Path with some utils
import rich
from fastcore.test import *  # type: ignore
from loguru import logger


# Helpers

In [ ]:
# pretty print by default
# %load_ext rich

In [ ]:

console = Console(width=104, tab_size=4, force_jupyter=True)
cprint = console.print


## OCR results clean-up

In [ ]:
#| export

def remove_multiple_whitespaces(text):
    return ' '.join(text.split())

    
def postprocess_ocr(text):
    "Basic postprocessing for English Tesseract OCR results."
    return ' '.join(remove_multiple_whitespaces(text).splitlines()).capitalize()


## OCR metric
> Some basic ways to compare OCR results

In [ ]:
#| export

def accuracy_ocr_naive(text, ground_truth):
    return sum(1 for a, b in zip(text, ground_truth) if a == b) / len(text)


def accuracy_ocr_difflib(text, ground_truth):
    """
    Calculates the OCR accuracy based on the similarity between the OCR text and the ground truth text,
    using difflib's SequenceMatcher to account for differences in a manner similar to git diffs.

    :param text: The OCR-generated text.
    :param ground_truth: The ground truth text.
    :return: A float representing the similarity ratio between the OCR text and the ground truth, 
            where 1.0 is identical.
    """
    # Initialize the SequenceMatcher with the OCR text and the ground truth
    matcher = difflib.SequenceMatcher(None, text, ground_truth)
    
    # Get the similarity ratio
    similarity_ratio = matcher.ratio()
    
    return similarity_ratio

In [ ]:
#| export

def get_text_diffs_html(str1, str2: str | None=None, ignore_align: bool = False):
    str2 = str2 or ''
    matcher = difflib.SequenceMatcher(None, str1, str2)
    html_str1, html_str2 = "", ""
    _ch ='⎕'  # ▿
    ch = f'&#x{ord(_ch):x};'
    span1_g = lambda l: f"<span style='color: green;'>{ch*l}</span>" if l > 0 else ""
    span1_r = lambda l: f"<span style='color: red;'>{ch*l}</span>" if l > 0 else ""
    span2 = lambda s: f"<span style='color: red;'>{html.escape(s)}</span>" if s else ""

    for opcode in matcher.get_opcodes():
        tag, i1, i2, j1, j2 = opcode
        if tag == "equal":
            html_str1 += html.escape(str1[i1:i2])
            html_str2 += html.escape(str2[j1:j2])
        elif tag == "replace":
            max_span = max(i2 - i1, j2 - j1)
            # str1_segment = str1[i1:i2].ljust(max_span)
            html_str1 += html.escape(str1[i1:i2]) + span1_g(max_span - (i2 - i1))
            html_str2 += span2(str2[j1:j2]) + (span1_r(max_span - (j2 - j1)) if not ignore_align else '')
        elif tag == "delete":
            deleted_segment = str1[i1:i2]
            html_str1 += html.escape(deleted_segment)
            if not ignore_align: html_str2 += span1_r(len(deleted_segment))
        elif tag == "insert":
            inserted_segment = str2[j1:j2].replace(" ", _ch)
            html_str1 += span1_g(len(inserted_segment))
            html_str2 += span2(inserted_segment)
    html_str1 = f"<div style='font-family: monospace; white-space: pre-wrap;'>{html_str1}</div>"
    html_str2 = f"<div style='font-family: monospace; white-space: pre-wrap;'>{html_str2}</div>"
    return html_str1, html_str2

def display_text_diffs(str1, str2):
    """
    Displays two strings one above the other, with differing characters highlighted in red in the 
    second string only, using difflib.SequenceMatcher to align the strings and ensure matching 
    sequences are vertically aligned.

    :param str1: The first string to compare.
    :param str2: The second string to compare.
    """
    html_str1, html_str2 = get_text_diffs_html(str1, str2)
    display(HTML(f"<pre style='font-size: 14px;'>{html_str1}<br/>{html_str2}</pre>"))

In [ ]:
str1 = "This is an awesome_ test string."*4
str2 = "This was an amazing test▿ spring."*4
display_text_diffs(str1, str2)


In [ ]:
str1 = "I was in a bad mood, and Curt sensed it immediately..."
str2 = "I was in a bad mood, and Curt sensed it immediately ..."
display_text_diffs(str1, str2)

# Colophon
----


In [ ]:
import fastcore.all as FC
from nbdev.export import nb_export


In [ ]:
if FC.IN_NOTEBOOK:
    nb_export('ocr_metric.ipynb', '../testbed')
